In [1]:
import sys
print(sys.executable)
print(sys.version)


/app/cloned_repo/LLM-World/.venv/bin/python
3.11.14 (main, Dec 17 2025, 21:07:37) [Clang 21.1.4 ]


In [14]:
from transformers import BitsAndBytesConfig
import os
from datasets import load_dataset

# Data

In [15]:
!pwd

/app/cloned_repo/LLM-World/Notebooks


In [21]:
trn_data = %pwd
trn_data = trn_data.replace('Notebooks','Files/training_set.jsonl')
val_data = trn_data.replace('training_set','validation_set')
print(trn_data)      
print(val_data)      
print(os.path.exists(trn_data))
print(os.path.exists(val_data))


/app/cloned_repo/LLM-World/Files/training_set.jsonl
/app/cloned_repo/LLM-World/Files/validation_set.jsonl
True
True


In [23]:
## Convert to huggingface dataset

data = load_dataset("json", data_files={
    "train":trn_data,
    "valid":val_data
})
data
                    

Generating train split: 409 examples [00:00, 8837.21 examples/s]
Generating valid split: 20 examples [00:00, 13704.64 examples/s]


DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 409
    })
    valid: Dataset({
        features: ['messages'],
        num_rows: 20
    })
})

# Model

In [ ]:
# https://huggingface.co/ibm-granite/granite-4.0-h-1b

In [11]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor

In [ ]:
# model_id = "ibm-granite/granite-vision-3.1-2b-preview"
model_id = "ibm-granite/granite-4.0-h-1b"